Import Packages

In [129]:
import pandas as pd
import numpy as np
import warnings

from datetime import datetime, timedelta
from IPython.display import display

warnings.filterwarnings("ignore")

Read Country Data

In [130]:
country_data_processed = pd.read_csv("Country_Processed_Table.csv")#read input data
country_data_processed.head()

,Country,Year,Country_Key,Continent,Region,Capital,Currency,Birth_Rate_Crude_Per_1000,Population_Total_Count,Death_Rate_Crude_Per_1000,GNI_Per_Capita_USD,Labour_Force_Female_Percent,Labour_Force_Total_Count,Age_Dependency_Ratio_Percent_Working_Age,Age_First_Marriage_Male,Age_First_Marriage_Female,Age_Dependency_Ratio_Old,Age_Dependency_Ratio_Young,Completeness_Birth_Registration_Percent
0,Canada,2005,0,North America,North America,Ottawa,CAD,10.6,32243753,7.1,34810,46.637376,17821886,44.495093,28.7,26.98,18.954219,25.540874,100.0
1,Canada,2006,1,North America,North America,Ottawa,CAD,10.9,32571174,7.0,38510,46.875172,18030705,44.236994,28.6,26.60,19.138346,25.098648,100.0
2,Canada,2007,2,North America,North America,Ottawa,CAD,11.2,32889025,7.1,43090,47.036713,18379605,44.034776,28.7,26.98,19.353443,24.681333,100.0
3,Canada,2008,3,North America,North America,Ottawa,CAD,11.3,33247118,7.2,45650,46.974754,18667574,43.933903,28.7,26.98,19.620978,24.312924,100.0
4,Canada,2009,4,North America,North America,Ottawa,CAD,11.3,33628895,7.1,43230,47.200377,18799155,43.976753,28.7,26.98,19.966597,24.010156,100.0


Create Fact Table and Add Month Data

In [131]:
fact_table = pd.DataFrame()

month_data_processed = pd.read_csv("Month_Processed_Table.csv")#read input data

fact_table = month_data_processed.merge(country_data_processed, on="Year")
fact_table = fact_table.sort_values('Country')


Join with Population Data

In [132]:
population_data_processed = pd.read_csv("Pop_Processed_Table.csv")#read input data

fact_table = fact_table.merge(population_data_processed, left_on=['Country','Year' ], right_on=['Country','Year' ], how='left' )

fact_table.head()

,Month_Key,Name,Quarter,Year,Decade,Country,Country_Key,Continent,Region,Capital,...,Population_Growth_Percent,Population_Female_Percent,Population_Male_Percent,Rural_Population_Percent,Rural_Population_Growth_Percent,Urban_Population_Percent,Urban_Population_Growth_Percent,Life_Exp_Birth_Total_Yr,Life_Exp_Birth_Male_Yr,Life_Exp_Birth_Female_Yr
0,165,October,4,2018,21,Bolivia,109,South America,South America,"Sucre, La Paz",...,1.421891,49.783399,50.216601,30.575,0.299854,69.425,1.920061,71.239,68.434,74.209
1,174,July,3,2019,21,Bolivia,110,South America,South America,"Sucre, La Paz",...,1.399133,49.799706,50.200294,30.227,0.254399,69.773,1.899151,71.513,68.684,74.508
2,39,April,2,2008,20,Bolivia,99,South America,South America,"Sucre, La Paz",...,1.691118,49.668326,50.331674,34.465,0.385311,65.535,2.384748,66.937,64.656,69.334
3,110,March,1,2014,21,Bolivia,105,South America,South America,"Sucre, La Paz",...,1.544977,49.723902,50.276098,31.953,0.461741,68.047,2.057712,69.891,67.271,72.655
4,17,June,2,2006,20,Bolivia,97,South America,South America,"Sucre, La Paz",...,1.751711,49.659745,50.340255,35.372,0.465219,64.628,2.462901,65.853,63.680,68.137


Remove Unecessary Columns and Add Measures

In [134]:
#keep only keys and columns needed for joins
fact_table = pd.DataFrame(fact_table, columns=['Month_Key', "Country_Key", "Population_Key", "Country", "Year", "Name"])

population_human_development_raw = pd.read_csv("human_development_index_raw.csv")#read input data


countries = ["Canada", "United States", "Mexico", "Burundi", "Chad", "Niger", "Bolivia", "Philippines", "Indonesia"]

population_human_development_processed = population_human_development_raw[population_human_development_raw["Entity"].isin(countries)]
population_human_development_processed = population_human_development_processed[population_human_development_processed["Year"] >2004]
population_human_development_processed = population_human_development_processed.drop(["Code"], axis=1)

country = "Canada"
for country in countries:
    for i in range(3):
        country_data = {}
        country_data["Entity"] = country
        country_data["Year"] = 2018+i
        population_human_development_processed = population_human_development_processed.append(pd.DataFrame([country_data]), ignore_index=True)


population_human_development_processed['Entity'].replace({"United States": "USA"}, inplace = True)
population_human_development_processed = population_human_development_processed.sort_values(["Entity","Year"], ascending=[True,True])
population_human_development_processed.fillna(method="ffill", inplace=True)#perhaps change to extrapolation

fact_table = fact_table.merge(population_human_development_processed, left_on=['Country','Year' ], right_on=['Entity','Year' ], how='left').drop(columns=["Entity"])
fact_table = fact_table.sort_values(["Country","Year"], ascending=[True,True])
display(fact_table.head(50))
print(fact_table.shape)

,Month_Key,Country_Key,Population_Key,Country,Year,Name,Human Development Index (UNDP)
0,6,96,96,Bolivia,2005,July,0.624
1,2,96,96,Bolivia,2005,March,0.624
2,7,96,96,Bolivia,2005,August,0.624
3,4,96,96,Bolivia,2005,May,0.624
4,3,96,96,Bolivia,2005,April,0.624
5,5,96,96,Bolivia,2005,June,0.624
6,8,96,96,Bolivia,2005,September,0.624
7,10,96,96,Bolivia,2005,November,0.624
8,0,96,96,Bolivia,2005,January,0.624
9,11,96,96,Bolivia,2005,December,0.624


(1728, 7)
